In [ ]:
from functools import reduce
import threading
import multiprocessing as mp
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error
from ast import literal_eval
import datetime
from datetime import datetime, timedelta
import time
import pytz
import json
import math
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from tick.plot import plot_hawkes_kernels, plot_point_process
from tick.hawkes import HawkesExpKern, HawkesADM4, SimuHawkesExpKernels

In [ ]:
dataset = pd.read_csv(str(input()))
dataset

In [ ]:
#dataset.drop(['D'], axis=1, inplace = True)
#dataset

In [ ]:
timestampColumns = ['A', 'C', 'L']

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
dayAfterEndDate = float((time.mktime(endDate.timetuple()) / 3600) - (time.mktime(startDate.timetuple()) / 3600))
dayAfterEndDate

In [ ]:
def calculateActualIntensities(timestampList):
    result = {}
    for timePoint in timestampList:
        if math.floor(timePoint) not in result.keys():
            result[math.floor(timePoint)] = 0
        result[math.floor(timePoint)] = result[math.floor(timePoint)] + 1
    
    return result

for column in timestampColumns:
    dataset[column] = dataset[column].apply(lambda x: literal_eval(x))
    dataset[column] = dataset[column].apply(sorted)
    dataset['actual_intensities_' + column] = dataset[column].apply(calculateActualIntensities)

dataset

In [ ]:
def combineColumns(df, columns):
    return [np.array(df[column], dtype=np.double) for column in columns]

def combineDimensions(df, timestampDimensions):
    result = []
    for timestampDimension in timestampDimensions:
        result = result + df[timestampDimension]
    
    return np.sort(np.array(result, dtype=np.double))

dataset['timestamps'] = dataset.apply(lambda df: combineColumns(df, timestampColumns), axis = 1)
dataset['timestamps_train'] = dataset.apply(lambda df: [combineDimensions(df, timestampColumns)], axis = 1)
dataset

In [ ]:
def estimateBestDecay(timestamps):
    highestScore = None
    bestDecay = None
    for d in range(1, 11):
        learner = HawkesExpKern(decays=d, penalty='elasticnet', elastic_net_ratio=0.5)
        learner.fit(timestamps)
        score = learner.score(timestamps, end_times=dayAfterEndDate)
        if((highestScore == None) or (score >= highestScore)):
            highestScore = score
            bestDecay = d
    
    return bestDecay

In [ ]:
try:
    decays = pd.read_csv(str(input()))
    dataset = dataset.merge(decays, on=['metadata_context_id', 'metadata_user_id'])
except:
    dataset['decay'] = dataset['timestamps_train'].apply(estimateBestDecay)
    dataset[['metadata_context_id', 'metadata_user_id', 'decay']].to_csv('UnivariateHawkesDecays_Intercession_2022.csv', header = True, index = False)

dataset

In [ ]:
def getHawkesProcessExpKernAdjacency(df):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    return learner.adjacency

dataset['adjacency'] = dataset.apply(getHawkesProcessExpKernAdjacency, axis = 1)
dataset

In [ ]:
def getHawkesProcessExpKernBaseline(df):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    return learner.baseline

dataset['baseline'] = dataset.apply(getHawkesProcessExpKernBaseline, axis = 1)
dataset

In [ ]:
def getHawkesProcessExpKernIntensities(df, dim):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    intensities, intensity_times = learner.estimated_intensity([df['timestamps'][dim]], intensity_track_step=1, end_time=dayAfterEndDate)
    
    return intensities

for i in range(0, len(timestampColumns)):
    dataset['intensities_' + timestampColumns[i]] = dataset.apply(lambda df: getHawkesProcessExpKernIntensities(df, i), axis = 1)

dataset

In [ ]:
def getHawkesProcessExpKernIntensityTimes(df, dim):
    learner = HawkesExpKern(decays=df['decay'], penalty='elasticnet', elastic_net_ratio=0.5)
    learner.fit(df['timestamps_train'])
    intensities, intensity_times = learner.estimated_intensity([df['timestamps'][dim]], intensity_track_step=1, end_time=dayAfterEndDate)
    
    return intensity_times

for i in range(0, len(timestampColumns)):
    dataset['intensity_times_' + timestampColumns[i]] = dataset.apply(lambda df: getHawkesProcessExpKernIntensityTimes(df, i), axis = 1)

dataset

In [ ]:
def splitResultsExpKern(overallResult, dimensions):
    for i in range(0, len(dimensions)):
        overallResult['baseline_' + dimensions[i]] = overallResult['baseline'].apply(lambda b: b.tolist()[i])
        
        for j in range(0, len(dimensions)):
            overallResult['influence_' + dimensions[i] + dimensions[j]] = overallResult['adjacency'].apply(lambda a: a.tolist()[i][j])
    
    overallResult.drop(['baseline', 'adjacency'], axis = 1, inplace = True)
    return overallResult

dataset = splitResultsExpKern(dataset, ['U'])
dataset

In [ ]:
for timestampColumn in timestampColumns:
    dataset['intensities_' + timestampColumn] = dataset['intensities_' + timestampColumn].apply(lambda x: x[0].tolist())
    dataset['intensity_times_' + timestampColumn] = dataset['intensity_times_' + timestampColumn].apply(lambda x: x.tolist())
    dataset['intensities_' + timestampColumn] = dataset.apply(lambda df: dict(zip(df['intensity_times_' + timestampColumn], df['intensities_' + timestampColumn])), axis = 1)

dataset

In [ ]:
def fixTimestamps(df, column):
    result = {}
    for h in df[column].keys():
        if math.floor(h) not in result.keys():
            result[math.floor(h)] = 0
        result[math.floor(h)] = result[math.floor(h)] + df[column][h]
    return result

for dim in timestampColumns:
    dataset['intensities_' + dim] = dataset.apply(lambda df: fixTimestamps(df, 'intensities_' + dim), axis = 1)

dataset

In [ ]:
def completeTimestamps(df, colName):
    result = {}
    for t in range(0, math.floor(dayAfterEndDate) + 1):
        if t not in df[colName].keys():
            result[t] = 0
        else:
            result[t] = df[colName][t]
    return result

for colName in ['actual_intensities_', 'intensities_']:
    for dim in timestampColumns:
        dataset[colName + dim] = dataset.apply(lambda df: completeTimestamps(df, colName + dim), axis = 1)

dataset

In [ ]:
modalities = dataset['Modality'].unique()

In [ ]:
def combineIntensityDictionaries(x, y):
    combinedDictionary = {}
    
    for key in x.keys():
        if key not in combinedDictionary.keys():
            combinedDictionary[key] = 0
        combinedDictionary[key] = combinedDictionary[key] + x[key]
    for key in y.keys():
        if key not in combinedDictionary.keys():
            combinedDictionary[key] = 0
        combinedDictionary[key] = combinedDictionary[key] + y[key]
    
    return combinedDictionary
    

def sumIntensities(series):
    return reduce(combineIntensityDictionaries, series)

intensityColumns = {}
for dimension in timestampColumns:
    intensityColumns['actual_intensities_' + dimension] = sumIntensities
    intensityColumns['intensities_' + dimension] = sumIntensities

intensityPlotDataset = dataset.groupby(['Modality']).agg(intensityColumns).reset_index()
intensityPlotDataset

In [ ]:
courseStudentCount = dataset.groupby(['Modality'])['course_code'].count().to_frame('total').reset_index()
courseStudentCount

In [ ]:
intensityPlotDataset = intensityPlotDataset.merge(courseStudentCount, on=['Modality'])
intensityPlotDataset

In [ ]:
def getAverageIntensities(df, column):
    result = {}
    for k in sorted([key for key in df[column].keys()]):
        result[k] = df[column][k] / df['total']
    return result

for column in ['actual_intensities_' + dimension for dimension in timestampColumns] + ['intensities_' + dimension for dimension in timestampColumns]:
    intensityPlotDataset[column] = intensityPlotDataset.apply(lambda df: getAverageIntensities(df, column), axis = 1)

intensityPlotDataset

In [ ]:
for modality in modalities:
    fig, axs = plt.subplots(len(timestampColumns), 1)
    intensityPlotDatasetModality = intensityPlotDataset.loc[intensityPlotDataset['Modality'] == modality]
    
    for i in range(0, len(timestampColumns)):
        axs[i].plot([t for t in intensityPlotDatasetModality.iloc[0]['actual_intensities_' + timestampColumns[i]].keys()], [v for v in intensityPlotDatasetModality.iloc[0]['actual_intensities_' + timestampColumns[i]].values()], color='tab:blue')
        axs[i].plot([t for t in intensityPlotDatasetModality.iloc[0]['intensities_' + timestampColumns[i]].keys()], [v for v in intensityPlotDatasetModality.iloc[0]['intensities_' + timestampColumns[i]].values()], color='tab:orange')
        axs[i].set_title(timestampColumns[i])
    
    for ax in axs.flat:
        ax.set(xlabel='Time', ylabel='Intensity')
    
    fig.legend(handles=[Line2D([0], [0], color='tab:blue', label='Actual Intensities'), Line2D([0], [0], color='tab:orange', label='Est. Intensities')], loc='lower center')
    fig.tight_layout(pad=1.0)
    fig.set_figwidth(10)
    fig.set_figheight(10)
    fig.savefig(modality + '_Intensities_Univariate.png')
    plt.close()

In [ ]:
for dim in timestampColumns:
    dataset['rmse_' + dim] = dataset.apply(lambda df: mean_squared_error([v for v in df['actual_intensities_' + dim].values()], [v for v in df['intensities_' + dim].values()], squared=False), axis = 1)

dataset

In [ ]:
averageAgg = {}
for dim in timestampColumns:
    averageAgg['rmse_' + dim] = np.mean

rmseModalities = dataset.groupby(['Modality']).agg(averageAgg).reset_index()
rmseModalities

In [ ]:
rmseModalities.to_csv('RMSE_Univariate.csv', header = True, index = False)